In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
VWAP 3-Candle Intraday Strategy Backtester (NSE, yfinance)
HARD-CODED: fetch exactly the last 60 days of intraday data (period="60d").

Features:
- Fixed capital_per_trade = ₹1,00,000 (pre-leverage), leverage default 5x
- Absolute rupee exits (SL ₹500, TP ₹2,000) at position level
- Optional trailing SL (rupees)
- Direction toggles (enable_longs / enable_shorts)
- Groww Intraday (NSE) cost model applied (brokerage, STT, stamp, exchange, SEBI, IPFT, GST)
- One trade per day per symbol, next-bar open execution, intraday square-off

Outputs:
- ./outputs_vwap/trades.csv
- ./outputs_vwap/summary.csv (per-ticker + overall, includes ROC on deployed capital)
"""

import math, os
from dataclasses import dataclass
from typing import List, Dict, Optional

import numpy as np
import pandas as pd

# =========================
# -------- CONFIG ---------
# =========================

@dataclass
class Config:
    tickers: List[str] = None
    interval: str = "5m"              # works with 60d period
    timezone: str = "Asia/Kolkata"

    # Entry window (IST)
    ENTRY_START_IST: str = "09:20"
    ENTRY_END_IST:   str = "14:30"
    SQUARE_OFF_IST:  str = "15:25"

    # Strategy toggles
    enable_longs: bool  = True
    enable_shorts: bool = True
    max_distance_pct: float = 0.02  # None to disable proximity-to-VWAP filter

    # Capital & risk
    capital_per_trade: float = 100000.0  # fixed allocation per trade
    leverage: float          = 5.0
    rupee_stop_loss: float   = 500.0
    rupee_take_profit: float = 2000.0
    enable_trailing_sl: bool = False
    trail_rupees: float      = 500.0

    # I/O
    output_dir: str = "./outputs_vwap"
    random_seed: int = 42

# =========================
# ---- COSTS (GROWW) ------
# =========================

def _brokerage(turnover_side: float) -> float:
    # Brokerage per side: max(min(₹20, 0.1% of turnover), ₹5)
    return max(min(0.001 * turnover_side, 20.0), 5.0)

def _exchange_charges(turnover_side: float) -> float:
    # 0.00297% per side
    return 0.0000297 * turnover_side

def _sebi_ipft(turnover_side: float) -> float:
    # SEBI 0.0001% + IPFT 0.0001% = 0.0002% per side
    return 0.000002 * turnover_side

def _gst(brokerage: float, exch: float, sebi_ipft: float) -> float:
    return 0.18 * (brokerage + exch + sebi_ipft)

def costs_on_side(turnover_side: float, side: str) -> float:
    brk  = _brokerage(turnover_side)
    exch = _exchange_charges(turnover_side)
    sip  = _sebi_ipft(turnover_side)
    gstv = _gst(brk, exch, sip)
    stt = 0.0
    stamp = 0.0
    if side == "sell":
        stt = 0.00025 * turnover_side           # 0.025% sell
    elif side == "buy":
        stamp = 0.00003 * turnover_side         # 0.003% buy
    return brk + exch + sip + gstv + stt + stamp

def full_round_trip_costs(buy_turnover: float, sell_turnover: float) -> float:
    return costs_on_side(buy_turnover, "buy") + costs_on_side(sell_turnover, "sell")

# =========================
# ---- DATA & HELPERS -----
# =========================

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def to_ist(df: pd.DataFrame, tz: str) -> pd.DataFrame:
    if df.index.tz is None:
        df = df.tz_localize("UTC")
    return df.tz_convert(tz)

def compute_session_vwap(df: pd.DataFrame, tz: str) -> pd.Series:
    d = df.copy()
    d["date"] = d.index.tz_convert(tz).date
    tp = (d["High"] + d["Low"] + d["Close"]) / 3.0
    d["tpv"] = tp * d["Volume"].fillna(0)
    d["cum_tpv"] = d.groupby("date")["tpv"].cumsum()
    d["cum_vol"] = d.groupby("date")["Volume"].cumsum()
    return d["cum_tpv"] / d["cum_vol"].replace(0, np.nan)

def within_time(t, start_str, end_str) -> bool:
    ts = pd.Timestamp(t).tz_convert("Asia/Kolkata")
    st = pd.Timestamp(f"{ts.date()} {start_str}", tz="Asia/Kolkata")
    en = pd.Timestamp(f"{ts.date()} {end_str}",   tz="Asia/Kolkata")
    return (ts >= st) and (ts <= en)

def before_time(t, cutoff_str) -> bool:
    ts = pd.Timestamp(t).tz_convert("Asia/Kolkata")
    co = pd.Timestamp(f"{ts.date()} {cutoff_str}", tz="Asia/Kolkata")
    return ts <= co

def next_bar_open(idx: pd.DatetimeIndex, i: int) -> Optional[pd.Timestamp]:
    return idx[i + 1] if i + 1 < len(idx) else None

# =========================
# --- STRATEGY ENGINE -----
# =========================

def scan_three_candle_entries(day_df: pd.DataFrame, vwap: pd.Series, cfg: Config) -> Optional[dict]:
    df = day_df.copy()
    df["VWAP"] = vwap.loc[df.index]

    for i in range(len(df) - 3):
        t_i = df.index[i]
        if not within_time(t_i, cfg.ENTRY_START_IST, cfg.ENTRY_END_IST):
            continue

        close_i = df["Close"].iloc[i]
        vwap_i  = df["VWAP"].iloc[i]

        long_open  = (close_i > vwap_i) and cfg.enable_longs
        short_open = (close_i < vwap_i) and cfg.enable_shorts
        if not (long_open or short_open):
            continue

        high_i, low_i = df["High"].iloc[i], df["Low"].iloc[i]

        j = i + 1
        if j >= len(df): break
        high_j, low_j = df["High"].iloc[j], df["Low"].iloc[j]

        got_signal = False
        dirn = None
        if long_open and (high_j > high_i):
            got_signal = True; dirn = "long"
        elif short_open and (low_j < low_i):
            got_signal = True; dirn = "short"
        if not got_signal:
            continue

        k = j + 1
        if k >= len(df): break
        high_k, low_k = df["High"].iloc[k], df["Low"].iloc[k]

        # Invalidate if opposite side breaks first
        if dirn == "long":
            if low_k < low_j:  # opposite break
                continue
            if high_k <= high_j:
                continue
        else:
            if high_k > high_j:
                continue
            if low_k >= low_j:
                continue

        entry_ts = next_bar_open(df.index, k)
        if entry_ts is None:
            continue
        if not within_time(entry_ts, cfg.ENTRY_START_IST, cfg.ENTRY_END_IST):
            continue

        entry_price = df.loc[entry_ts, "Open"]

        # Optional proximity filter
        if cfg.max_distance_pct is not None:
            vwap_k = df["VWAP"].iloc[k]
            dist = abs(entry_price - vwap_k) / vwap_k
            if dist > cfg.max_distance_pct:
                continue

        return {
            "direction": dirn,
            "entry_ts": entry_ts,
            "entry_price": float(entry_price),
            "open_idx": i,
            "signal_idx": j,
            "entry_idx": k
        }

    return None

def simulate_trade(day_df: pd.DataFrame, entry: dict, cfg: Config) -> dict:
    direction, entry_ts, entry_price = entry["direction"], entry["entry_ts"], entry["entry_price"]

    # Position sizing off FIXED capital per trade
    notional = cfg.capital_per_trade * cfg.leverage
    qty = math.floor(notional / entry_price)
    if qty <= 0:
        return {"skipped": True, "reason": "qty_zero"}

    buy_turnover = 0.0
    sell_turnover = 0.0
    if direction == "long":
        buy_turnover += qty * entry_price
    else:
        sell_turnover += qty * entry_price  # short opens on sell

    idx = day_df.index
    start_pos = idx.get_loc(entry_ts)

    best_run_pnl = 0.0
    trail_stop = None

    exit_reason = "square_off"
    exit_ts = None
    exit_price = None

    def position_pnl(price_now: float) -> float:
        return (price_now - entry_price) * qty if direction == "long" else (entry_price - price_now) * qty

    for i in range(start_pos, len(idx)):
        ts = idx[i]
        if not before_time(ts, cfg.SQUARE_OFF_IST):
            exit_ts = ts
            exit_price = float(day_df.loc[ts, "Open"])
            exit_reason = "square_off"
            break

        close_price = float(day_df.loc[ts, "Close"])
        run_pnl = position_pnl(close_price)

        if cfg.enable_trailing_sl and run_pnl > best_run_pnl:
            best_run_pnl = run_pnl
            trail_stop = max(best_run_pnl - cfg.trail_rupees, 0.0)

        hit_tp = (run_pnl >= cfg.rupee_take_profit)
        hit_sl = (run_pnl <= -cfg.rupee_stop_loss)
        hit_trail = (cfg.enable_trailing_sl and trail_stop is not None and run_pnl <= trail_stop)

        if hit_tp or hit_sl or hit_trail:
            nxt = next_bar_open(idx, i)
            if nxt is None:
                exit_ts = ts
                exit_price = float(day_df.loc[ts, "Open"])
            else:
                exit_ts = nxt
                exit_price = float(day_df.loc[nxt, "Open"])
            exit_reason = "target_hit" if hit_tp else ("stop_hit" if hit_sl else "trail_hit")
            break

    if exit_ts is None:
        last_ts = idx[-1]
        exit_ts = last_ts
        exit_price = float(day_df.loc[last_ts, "Open"])
        exit_reason = "eod_forced"

    if direction == "long":
        sell_turnover += qty * exit_price
    else:
        buy_turnover += qty * exit_price

    gross_pnl = (exit_price - entry_price) * qty if direction == "long" else (entry_price - exit_price) * qty
    fees = full_round_trip_costs(buy_turnover, sell_turnover)
    net_pnl = gross_pnl - fees

    return {
        "skipped": False,
        "symbol": day_df.attrs.get("symbol",""),
        "direction": direction,
        "entry_ts": entry_ts,
        "entry_price": entry_price,
        "exit_ts": exit_ts,
        "exit_price": exit_price,
        "qty": qty,
        "buy_turnover": buy_turnover,
        "sell_turnover": sell_turnover,
        "gross_pnl": gross_pnl,
        "fees": fees,
        "net_pnl": net_pnl,
        "exit_reason": exit_reason,
        "capital_per_trade": cfg.capital_per_trade,
        "leverage": cfg.leverage
    }

# =========================
# --------- RUNNER --------
# =========================

def fetch_yf_60d(tickers: List[str], interval: str) -> Dict[str, pd.DataFrame]:
    # Hard-coded 60d period to align with intraday data retention limits
    import yfinance as yf
    data = {}
    for t in tickers:
        df = yf.download(t, period="60d", interval=interval, progress=False, auto_adjust=False, multi_level_index=False)
        if df is None or len(df) == 0:
            continue
        data[t] = df
    return data

def main():
    cfg = Config(
        tickers=["RELIANCE.NS","TCS.NS","HDFCBANK.NS"],
        interval="5m",  # yfinance supports 1m (30d), 2m/5m (60d), 15m (60d)
        enable_longs=True,
        enable_shorts=True,
        capital_per_trade=100000.0,
        leverage=5.0,
        rupee_stop_loss=500.0,
        rupee_take_profit=2000.0,
        enable_trailing_sl=True,
        trail_rupees=600.0,
        max_distance_pct=0.02,
        output_dir="./outputs_vwap",
    )

    np.random.seed(cfg.random_seed)
    ensure_dir(cfg.output_dir)

    print(f"Downloading last 60d for {len(cfg.tickers)} tickers …")
    raw = fetch_yf_60d(cfg.tickers, cfg.interval)

    all_trades = []
    for sym, df in raw.items():
        if df is None or len(df) == 0:
            continue

        df = to_ist(df, cfg.timezone)
        df = df.rename(columns={c: c.title() for c in df.columns})
        df.attrs["symbol"] = sym

        vwap = compute_session_vwap(df, cfg.timezone)
        df["session_date"] = df.index.tz_convert(cfg.timezone).date

        for d, day_df in df.groupby("session_date"):
            day_df = day_df.drop(columns=["session_date"]).copy()
            entry = scan_three_candle_entries(day_df, vwap, cfg)
            if not entry:
                continue
            tr = simulate_trade(day_df, entry, cfg)
            if not tr.get("skipped", False):
                all_trades.append(tr)

    if len(all_trades) == 0:
        print("No trades generated in last 60d. Try disabling proximity filter or using 5m bars.")
        return

    trades = pd.DataFrame(all_trades)
    trades["entry_ts"] = pd.to_datetime(trades["entry_ts"]).dt.tz_convert(cfg.timezone)
    trades["exit_ts"]  = pd.to_datetime(trades["exit_ts"]).dt.tz_convert(cfg.timezone)
    trades = trades.sort_values(["symbol","entry_ts"]).reset_index(drop=True)

    # Per-ticker summary + ROC
    summary = trades.groupby("symbol").agg(
        n_trades=("symbol","count"),
        wins=("net_pnl", lambda s: int((s > 0).sum())),
        losses=("net_pnl", lambda s: int((s <= 0).sum())),
        gross_pnl=("gross_pnl","sum"),
        fees=("fees","sum"),
        net_pnl=("net_pnl","sum"),
        avg_net=("net_pnl","mean"),
    ).reset_index()
    summary["win_rate"] = (summary["wins"] / summary["n_trades"]).round(3)
    summary["capital_deployed"] = summary["n_trades"] * cfg.capital_per_trade
    summary["roc_on_deployed"] = (summary["net_pnl"] / summary["capital_deployed"]).fillna(0.0)

    # Overall row
    total_trades = int(trades.shape[0])
    overall = pd.DataFrame([{
        "symbol": "ALL",
        "n_trades": total_trades,
        "wins": int((trades["net_pnl"] > 0).sum()),
        "losses": int((trades["net_pnl"] <= 0).sum()),
        "gross_pnl": trades["gross_pnl"].sum(),
        "fees": trades["fees"].sum(),
        "net_pnl": trades["net_pnl"].sum(),
        "avg_net": trades["net_pnl"].mean(),
        "win_rate": (trades["net_pnl"] > 0).mean(),
        "capital_deployed": total_trades * cfg.capital_per_trade,
        "roc_on_deployed": (trades["net_pnl"].sum() / (total_trades * cfg.capital_per_trade)) if total_trades > 0 else 0.0
    }])

    summary_out = pd.concat([summary, overall], ignore_index=True)

    # Save
    trades_csv  = os.path.join(cfg.output_dir, "trades.csv")
    summary_csv = os.path.join(cfg.output_dir, "summary.csv")
    trades.to_csv(trades_csv, index=False)
    summary_out.to_csv(summary_csv, index=False)

    print(f"\nSaved trades:  {trades_csv}")
    print(f"Saved summary: {summary_csv}")

if __name__ == "__main__":
    main()



Saved trades:  ./outputs_vwap/trades.csv
Saved summary: ./outputs_vwap/summary.csv
